This notebook should be used after the notebook Read_files_create_batch.

The purpose of this notebook is to generate a 2D projection that fits your needs. 

        
If you use 3D images, you will have to choose between doing a maximal intensity projection notebook or a .

 The simple max projection notebook makes a maximal intensity
    projectection (if you do segmentation of the cell body using the FISH data, this is for sure the best option). 
    If you do a segmentation of the nuclei and of the cell body using the DAPI (+cell mask) you should to use the Special intensity projection.
    
    With these notebook you will generate a 2D maximal intensity projection.

If you have 2D images,  this notebook will not do any processing, but will copy images to another folder (for consistency among cases), and will store 
in the right variables the file addresses.  

In complement, at the end of the notebook, you can if needed try the histogram equalization on maximal intensity projection images, in order to enhance the cytoplasm signal (DAPI + CellMask).

23/04/25     Jacques Bourg @ Florian Muller lab. Institut Pasteur.


<div style="background-color: white; padding: 10px;">
    <img src="./pipeline.png" alt="pipeline" width="1200" height="420">
</div>

In [ ]:
import os
import sys
import numpy as np
from pathlib import Path
import skimage.io as io
from skimage.transform import rescale 
from skimage.exposure import equalize_hist
import napari
import ipywidgets as widgets
from IPython.display import display

from apifish.stack import get_in_focus_indices, compute_focus

In [ ]:
%load_ext autoreload
%autoreload 2
    
base_dir = Path("../../pipeline/src").resolve()
sys.path.append(str(base_dir))
sys.path.append(str(base_dir / "utils"))
        
from utils.parameters_tracking import Parameter_tracking as Track
from utils.plots import Plots
tk   = Track()
pts  = Plots()

In [ ]:
var = str(Path('../Analysis'))
batch_folders = os.listdir(var)
dropdown = widgets.Dropdown(options=batch_folders, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown)

In [ ]:
n         = np.where(np.array(batch_folders) == dropdown.value)[0][0]
file_path = str(Path(var) / Path(batch_folders[n]) / Path(batch_folders[n] +'.json'))
constants = tk.load_json(file_path)
batch_name= constants['BATCH_NAME']; print(batch_name)

In [ ]:
modalities = constants['MODALITIES']
dropdown2 = widgets.Dropdown(options=modalities, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown2)

In [ ]:
n2  = np.where(np.array(modalities) == dropdown2.value)[0][0]
modality = modalities[n2];print(modality)

In [ ]:
channel = constants['CHANNELS']
dropdown3 = widgets.Dropdown(options=channel, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown3)

In [ ]:
n3      = np.where(np.array(channel) == dropdown3.value)[0][0]
chan    = channel[n3];print(chan)

In [ ]:
structs   = constants['STRUCTURES']
dropdown4 = widgets.Dropdown(options=structs, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown4)

In [ ]:
n4   = np.where(np.array(structs) == dropdown4.value)[0][0]
struc = structs[n4]; print(struc)

In [ ]:
batch_mod_chan        = constants[f'BATCH_{modality}_{chan}']
folder_struc          = Path(f"../Analysis/{batch_name}/{modality}/{chan}/{struc}/")
if not folder_struc.exists():
    folder_struc.mkdir(parents=True)

folder_mod_chan_mip = Path(f"../Analysis/{batch_name}/{modality}/{chan}/{struc}/train_2D")
if not folder_mod_chan_mip.exists():
    folder_mod_chan_mip.mkdir(parents=True)

3 alternatives:

   ### A -
    Do a simple maximal intensity projection. Once this is done, go to the end of the notebook (to save the variables).
    This simpler method is more appropriated for segmenting the cellbody with FISH data.
   ### B -
     Do a special maximal intensity projection. Use this method by default (except when segmenting cell bodies with FISH data). 
     In this case, we compute the focus on each focal plane, applying Helmli and Scherer’s mean method 
     to determine the N planes with the best focus, and out of those planes, we compute the maximal intensity projection. 
     If you don't know if to do A or B, go to B, you will be able to compare both. If it turned out you prefered A, simply go back to A and 
     Once this is done, go to the end of the notebook (to save the variables).
   ### C -
     You have 2D images: go to C, at the end of the notebook.

 ## A

In [ ]:
batch_mod_chan_mip = []
viewer_mp = napari.Viewer(title=f" {modality} {chan} {struc} MIP")
count = 0
for file_path in batch_mod_chan:
    file_path = Path(file_path)
    im        = io.imread(file_path)
    mip_raw   = np.max(im, axis=0)
    new_file_name = '_'.join(Path(file_path).stem.split('_')[:-1]) + f'_MIP_{chan}_{struc}.tif'   # remove the last suffix (ie _FISH or _FISH1) puts another suffix _MIP and the channel
    new_file_add  = str(folder_mod_chan_mip / Path(new_file_name))
    io.imsave(new_file_add, mip_raw, imagej=True)
    batch_mod_chan_mip.append(new_file_add)
    val = np.percentile(mip_raw,99)
    viewer_mp.add_image(mip_raw, contrast_limits=(0, val), rgb=False, name=f"raw MIP {str(Path(str(file_path.parent.stem) +  '_'  + str(file_path.stem)))}")
    if count !=0:
        viewer_mp.layers[f"raw MIP {str(Path(str(file_path.parent.stem) +  '_'  + str(file_path.stem)))}"].visible  = False
    count = count + 1

 ## B
  Select planes that have the best focus an do a mip on those .We use a non linear colormap to enhance the effect of the processing.

In [ ]:

NEIGHBORHOOD_SIZE_FOCUS = 31  # parameter used for focus computation
cm                      = pts.custom_color_map_for_napari(pow=.5, nb_points= 256) # non linear colormap

batch_mod_chan_mip = []
viewer_mp_s = napari.Viewer(title=f" {modality} {chan} {struc} Special MIP")
count     = 0
for file_path in batch_mod_chan:

    file_path = Path(file_path)
    im = io.imread(file_path)
    PROPORTION_FOCUS = int(5*100/np.shape(im)[0]) 

    focus     = compute_focus(im, neighborhood_size=NEIGHBORHOOD_SIZE_FOCUS)
    inds      = get_in_focus_indices(focus, PROPORTION_FOCUS)
    im_sub    = im[np.array(inds, dtype=int),:,:]
    
    mip_im    = np.max(im_sub, axis=0)
    mip_raw   = np.max(im, axis=0)

    new_file_name = '_'.join(Path(file_path).stem.split('_')[:-1]) + f'_MIP_{chan}_{struc}.tif'   # remove the last suffix (ie _FISH or _FISH1) puts another suffix _MIP and the channel
    new_file_add  = str(folder_mod_chan_mip / Path(new_file_name))
    io.imsave(new_file_add, mip_im, imagej=True)
    batch_mod_chan_mip.append(new_file_add)
        
    mip_im_rescaled  = rescale(mip_im, 1.0, anti_aliasing=False)    
    mip_raw_rescaled = rescale(mip_raw, 1.0, anti_aliasing=False)       

    viewer_mp_s.add_image(mip_im_rescaled, colormap=('custom_gray', cm), name=f"Selected MIP {str(Path(str(file_path.parent.stem) +  '_'  + str(file_path.stem)))}")
    viewer_mp_s.add_image(mip_raw_rescaled, colormap=('custom_gray', cm), name=f"raw MIP {str(Path(str(file_path.parent.stem) +  '_'  + str(file_path.stem)))}")
    
    if count !=0:
        viewer_mp_s.layers[f"Selected MIP {str(Path(str(file_path.parent.stem) +  '_'  + str(file_path.stem)))}"].visible = False
        viewer_mp_s.layers[f"raw MIP {str(Path(str(file_path.parent.stem) +  '_'  + str(file_path.stem)))}"].visible      = False

    count = count+1

## C
Only 2D images: copy them in the corresponding folder ../Analysis/{batch_name}/{modality}/{chan}/train_2D  and fill the list batch_mod_chan_mip

In [ ]:
batch_mod_chan_mip = []
for file_path in batch_mod_chan: 
    file_path = Path(file_path)
    im = io.imread(file_path)
    new_file_name = '_'.join(Path(file_path).stem.split('_')[:-1]) + f'_MIP_{chan}_{struc}.tif'
    new_file_add  = str(folder_mod_chan_mip / Path(new_file_name))
    io.imsave(new_file_add, im, imagej=True)
    batch_mod_chan_mip.append(new_file_add)

Save parameters

In [ ]:
exec(f"BATCH_{modality}_{chan}_{struc}_MIP = batch_mod_chan_mip", globals())   ##### turn lower case variables into uppercase for tracking     

In [ ]:
constants2 = tk.collect_constants()
tk.save_constants_and_commit_hash(constants2, batch_name, folder_path = Path(f"../Analysis/{batch_name}"))

# F
### Complement : histogram equalization. 

In case we use urea with Cellmask, one must perform histogram equalization of the image in order to recover the cellmask
signal marking the cytoplasm, urea having a detrimental effect on the cell mask.

In this section you can evaluate whether that processing is suited for your DAPI data. In case it is, we will overwrite the data, in order to be fed to the Cellpose 
for training of the cell segmentation.

In [ ]:
viewer_eq      = napari.Viewer(title="Equalization of images")
list_images_eq = []
counter        = 0
for ind, file_name in enumerate(batch_mod_chan_mip):
    im     = io.imread(file_name)
    img_eq = equalize_hist(im)

    value_max1   = np.percentile(im, 99)
    value_max2   = np.percentile(img_eq, 99)

    viewer_eq.add_image(im, rgb=False, name=f"MIP {Path(file_name).stem}", contrast_limits=(0, value_max1))
    
    value_max2   = np.percentile(img_eq, 99)
    viewer_eq.add_image(img_eq, rgb=False, name=f"EQUALIZED {Path(file_name).stem}", contrast_limits=(0, value_max2))
                                                                           
    list_images_eq.append(img_eq)
    
    if counter != 0:
        viewer_eq.layers[f"MIP {Path(file_name).stem}"].visible = False
        viewer_eq.layers[f"EQUALIZED {Path(file_name).stem}"].visible = False
    counter += 1

In [ ]:
# in case you prefer this segmentation for the cell bodies: overwrite these images.
for ind, file_name in enumerate(batch_mod_chan_mip):
   io.imsave(file_name, np.asarray(list_images_eq[ind], dtype=np.float32) ,imagej=True)  

No need to save the constants, the image addresses are the same. 